# Flood Adaptation: Minimal Model

This python script outlines a basic Agent-Based Model (ABM) developed using the Mesa framework, designed as a simplified representation of household adaptation to flood events in a social network context. The model serves as a template for understanding the fundamental mechanics of flood adapdtation with a given social network.

This directory is structured as follows:
 - A Mesa Agent class `Households` is defined in `agents.py`
 - A Mesa Agent class `Government` is defined in `agents.py` (this agent currently does not perform any action)
 - A Mesa Model class `AdaptationModel` is defined in `model.py`
 - This notebook `demo.ipynb` provides a demonstration of the model and its visualization.

Each household agent in the model represents a household with attributes and behaviors that determine its response to flood events. The flood depth attribute assigned to each household is currently random for demonstration purposes. The decision by each household to adapt is also arbitrary. In a more comprehensive model, this could be replaced with more sophisticated decision-making algorithms.

The current implementation includes several simplifications and arbitrary choices, such as random flood depths and basic adaptation logic based on random thresholds. These aspects are designed for clarity and ease of understanding and should be replaced with realistic and sound choices.

You are encouraged to modify, expand, and customize the model. This might include integrating real-world data, implementing more complex decision-making processes for agents, or exploring different network topologies and their effects on social dynamics.

The visualisation provided is basic. Additional aspects can be added for visualization and more in-depth analysis.

_This notebook was tested on 2023-11-13 with Mesa version 2.1.4 on Python 3.12.0._

# Running the model and visualizing the model domain and social network over time

In [ ]:
from model import AdaptationModel
from agents import Government, Households
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from mesa.batchrunner import batch_run
import pandas as pd

# Initialize the Adaptation Model with 50 household agents.
model = AdaptationModel(number_of_households=25, flood_map_choice="harvey",network="watts_strogatz") # flood_map_choice can be "harvey", "100yr", or "500yr"

params = {
    "flood_warning":[1.5,1.5,1.5] #To run a base case multiple times
    #"flood_warning": [0, 0.05,0.1, 0.15, 0.2, 0.25, 0.8],
    # "discount_rate": [0.9, 0.97, 0.975, 0.98, 0.985, 0.99, 0.995],
    # "fine": [0,500,1000, 2000, 4000, 20000],
    #max_trust_value": [0.1,0.3, 0.8],
    #"elevation_costs_per_square_metre": [100,290,400,1500],    
        }

batch_run = batch_run(
        AdaptationModel,  
        parameters=params,
        iterations=5,  
        max_steps=80,  
        number_processes=1,
        data_collection_period=1,
        display_progress=True,
    )


# Calculate positions of nodes for the network plot.
# The spring_layout function positions nodes using a force-directed algorithm,
# which helps visualize the structure of the social network.
pos = nx.spring_layout(model.G)

# Define a function to plot agents on the network.
# This function takes a matplotlib axes object and the model as inputs.
def plot_network(ax, model):
    # Clear the current axes.
    ax.clear()
    # Determine the color of each node (agent) based on their adaptation status.
    for agent in model.schedule.agents:
        if isinstance(agent, Households):
            colors = 'blue' if agent.desire_to_take_measures else 'red' 
        #elif isinstance(agent, Government):
            #colors = 'green'
    # Draw the network with node colors and labels.
    nx.draw(model.G, pos, node_color=colors, with_labels=True, ax=ax)
    # Set the title of the plot with the current step number.
    ax.set_title(f"Social Network State at Step {model.schedule.steps}", fontsize=12)

# Generate the initial plots at step 0.
# Plot the spatial distribution of agents. This is a function written in the model.py
# model.plot_model_domain_with_agents()
# 
# # Plot the initial state of the social network.
# fig, ax = plt.subplots(figsize=(7, 7))
# plot_network(ax, model)
# plt.show()

# Run the model for 20 steps and generate plots every 5 steps.
for step in range(80):
    model.step()

    # Every 5 steps, generate and display plots for both the spatial distribution and network.
    # Note the first step is step 0, so the plots will be generated at steps 4, 9, 14, and 19, which are the 5th, 10th, 15th, and 20th steps.
    # if (step + 1) % 5 == 0:
        # Plot for the spatial map showing agent locations and adaptation status.
        # plt.figure(figsize=(10, 6))
        # model.plot_model_domain_with_agents()
        # 
        # # Plot for the social network showing connections and adaptation statuses.
        # fig, ax = plt.subplots(figsize=(7, 7))
        # plot_network(ax, model)
        # plt.show()

In [ ]:
agent_data = model.datacollector.get_agent_vars_dataframe()
agent_data


In [ ]:
model_data = model.datacollector.get_model_vars_dataframe()
model_data

In [ ]:
batch_data = pd.DataFrame(batch_run)
batch_data

In [ ]:
model_data = model.datacollector.get_model_vars_dataframe()
g = sns.lineplot(data=model_data)

In [ ]:
model_data = model.datacollector.get_model_vars_dataframe()
adapted_households = model_data['total_adapted_households'].values
g = sns.lineplot(data=adapted_households)
g.set(title="Adapted households over time", ylabel="Number of adapted households");

In [ ]:
agent_data = model.datacollector.get_agent_vars_dataframe()
perceived_flood_probability = agent_data['PerceivedFloodProbability'].values
g = sns.lineplot(data=perceived_flood_probability)
g.set(title="Perceived flood probability", ylabel="Perceived flood probability");

In [ ]:
plt.plot(model.average_perceived_flood_probability_over_time)
plt.xlabel('Time Step')
plt.ylabel('Average Perceived Flood Probability')
plt.title('Average Perceived Flood Probability Over Time')
plt.show()

In [ ]:
agent_data = model.datacollector.get_agent_vars_dataframe()
perceived_flood_probability = agent_data['FinedTotal'].values
g = sns.lineplot(data=perceived_flood_probability)
g.set(title="Total fined", ylabel="Fined in total");

In [ ]:
agent_data = model.datacollector.get_agent_vars_dataframe()
perceived_flood_probability = agent_data['PerceivedFloodProbability'].values
g = sns.histplot(perceived_flood_probability, discrete= False)
g.set(title="Perceived flood probability", ylabel="Number of timesteps");